# HW 2 - Naive Bayes in Hadoop MR
__`MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Fall 2018`__

In the live sessions for week 2 and week 3 you got some practice designing and debugging Hadoop Streaming jobs. In this homework we'll use Hadoop MapReduce to implement your first parallelized machine learning algorithm: Naive Bayes. As you develop your implementation you'll test it on a small dataset that matches the 'Chinese Example' in the _Manning, Raghavan and Shutze_ reading for Week 2. For the main task in this assignment you'll be working with a small subset of the Enron Spam/Ham Corpus. By the end of this assignment you should be able to:
* __... describe__ the Naive Bayes algorithm including both training and inference.
* __... perform__ EDA on a corpus using Hadoop MR.
* __... implement__ parallelized Naive Bayes.
* __... constrast__ partial, unordered and total order sort and their implementations in Hadoop Streaming.
* __... explain__ how smoothing affects the bias and variance of a Multinomial Naive Bayes model.

As always, your work will be graded both on the correctness of your output and on the clarity and design of your code. __Please refer to the `README` for homework submission instructions.__ 

## Notebook Setup
Before starting, run the following cells to confirm your setup.

In [1]:
# imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
# global vars (paths) - ADJUST AS NEEDED
JAR_FILE = "/usr/lib/hadoop-mapreduce/hadoop-streaming.jar"
HDFS_DIR = "/user/root/HW2"
HOME_DIR = "/media/notebooks/Assignments/HW2" # FILL IN HERE eg. /media/notebooks/Assignments/HW2

In [3]:
# save path for use in Hadoop jobs (-cmdenv PATH={PATH})
from os import environ
PATH  = environ['PATH']

In [4]:
# data path
ENRON = "data/enronemail_1h.txt"

# Question 1: Hadoop MapReduce Key Takeaways.  

This assignment will be the only one in which you use Hadoop Streaming to implement a distributed algorithm. The key reason we continue to teach Hadoop streaming is because of the way it forces the programmer to think carefully about what is happening under the hood when you parallelize a calculation. This question will briefly highlight some of the most important concepts that you need to understand about Hadoop Streaming and MapReduce before we move on to Spark next week.   

### Q1 Tasks:

* __a) short response:__ What "programming paradigm" is Hadoop MapReduce based on? What are the main ideas of this programming paradigm and how does MapReduce exemplify these ideas?

* __b) short response:__ What is the Hadoop Shuffle? When does it happen? Why is it potentially costly? Describe one specific thing we can we do to mitigate the cost associated with this stage of our Hadoop Streaming jobs.

* __c) short response:__ In Hadoop Streaming why do the input and output record format of a combiner script have to be the same? [__`HINT`__ _what level of combining does the framework guarantee? what is the relationship between the record format your mapper emits and the format your reducer expects to receive?_]

* __d) short response:__ To what extent can you control the level of parallelization of your Hadoop Streaming jobs? Please be specific.

* __e) short response:__ What change in the kind of computing resources available prompted the creation of parallel computation frameworks like Hadoop? 

### Q1 Student Answers:

> __a)__ Hadoop MapReduce is based on the functional programing paradigm. It is a declarative programming paradigm, which means programming is done with expressions. In functional code, the output value of a function depends only on the arguments that are input to the function. Key ideas in this paradigm include avoiding state changes/mutable data, and using higher-order functions (functions which take other functions as inputs). MapReduce exemplifies these ideas through two phases: 'map' which applies a transformation(or filter) to all elements of a collection, and 'reduce' which aggregates/folds the elements of a collection. Map and reduce are examples of higher-order functions.

> __b)__ Hadoop Shuffle is a process of transferring data from the mappers to reducers: the partitioner determines which reducer will be responsible for processing a particular key, and the execution framework uses this information to copy the data to the right location. It happens after 'map' phase and before 'reduce' phase. When data volume is large, it could lead to an excessive amount of across-the-network copy operations, as well as the time complexity of doing a merge sort at scale. We could apply local aggregation of intermediate results in memory in mappers or by using a combiner to reduce the amount of data to be transferred from mappers to reducers.  

> __c)__ Since Hadoop does not guarantee that a combiner will be executed, our record for- mat has to work whether or not it goes through the combiner. In other words, the signature of the input and output of the combiner must match the signature of the input to the reducer. 

> __d)__ We can control the level of parrallelization by specifying the number of reducers (through `-numReduceTasks` variable), but we cannot specify the number of mappers. The number of mappers is usually driven by the total size of the inputs, that is, the total number of blocks of the input files. The right level of parallelism for maps seems to be around 10-100 maps per-node, although it has been set up to 300 maps for very cpu-light map tasks. In terms of reducer, the ideal reducers should be the optimal value that gets them closest to:
   - Should process  multiples of the block size 
   - A task time between 5 and 15 minutes  
   - Creates the fewest files possible   

> __e)__ Both of the following two changes prompted the creation of parallel computation frameworks:
   - Opportunities for computing hardware improvement became diminishing. In addition, its returns no longer justified continued investment. 
   - Cheaper and cheaper hardware: rather than investing to build a super power computer, it is possible to and more sensible to build a network of less powerful computers. 
   - Substantial growth of large-data problems.

> **References**:
1. Jimmy Lin and Chris Dyer, Data-Intensive Text Processing with MapReduce. 
2. https://hadoop.apache.org/docs/r1.2.1/mapred_tutorial.html#Partitioner
3. https://techvidvan.com/tutorials/hadoop-mapreduce-shuffling-and-sorting/

# Question 2: MapReduce Design Patterns.  

In the last two live sessions and in your readings from Lin & Dyer you encountered a number of techniques for manipulating the logistics of a MapReduce implementation to ensure that the right information is available at the right time and location. In this question we'll review a few of the key techniques you learned.   

### Q2 Tasks:

* __a) short response:__ What are counters (in the context of Hadoop Streaming)? How are they useful? What kinds of counters does Hadoop provide for you? How do you create your own custom counter?

* __b) short response:__ What are composite keys? How are they useful? How are they related to the idea of custom partitioning?

* __c) short response:__ What is the order inversion pattern? What problem does it help solve? How do we implement it? 

### Q2 Student Answers:

> __a)__ Hadoop provides a lightweight API for constructs called “counters”, which can be used for counting events that occur during execution, e.g., number of corrupt records, number of times a certain condition is met, or anything that the programmer desires. It is a shared variable that is incremented and decremented atomically by the Hadoop framework. However, they are write-only variable and can only be read by log after the job is completed. They are useful in that 
   - (1) it provides a way to track the occurrences of global events within the map and reduce phases of jobs; 
   - (2) it is one of the easiest way to investigate internal behaviors of MapReduce programs.

> Hadoop built-in counter consists of three types:
   - MapReduce Task Counters: This counter collects tasks information over the course of their execution. Then aggregate results over all the tasks. For example, MAP_INPUT_RECORDS counter counts the input records read by each map task and aggregates over all map tasks in a job. 
   - File System Counter: File system counters track 2 main details, the number of bytes read by the file system and a number of bytes written.
   - Job Counter: Jobtracker maintain Job Counters, such as, TOTAL_LAUNCHED_MAPS. 

> We can create custom counter by specifying the following formatted line to the standard error stream: `reporter: counter: group, counter, amount`.  


> __b)__ Composite key contains more than one fields to form the key for Hadoop to shuffle. It is useful when we want to sort not just by the key but also by value, for instance, we might want to sort the words by their count to find the most frequent words but then break ties by the word in alphabetical order. By controlling how keys are sorted and how the key space is partitioned, we can present data to the reducer in the order necessary to perform the proper computations. 


> __c)__ Sometimes we need to calculate relative frequencies of co-occurring terms, which is calculated by $f(B|A)= \frac{count(A \cap B)}{count(A)}$. The “order inversion” design pattern is to control the order in which the total (count(A,$*$)) to come out first in the reducers stream by using a special key; so if the reducer receives this total first, then it can read each co-occurring pair and do the relative frequency calculation one by one. This design pattern resolve potential out-of-memory issue at the reducers or a second shuffle. To implement this, we need to get the word count first, and emit the key value of (A, $*$), because it will be alphabetically sorted before (A,aardvark) and other co-ocurring terms.

> **References**
1. Jimmy Lin and Chris Dyer, Data-Intensive Text Processing with MapReduce.
2. https://data-flair.training/forums/topic/what-is-counter-in-hadoop-mapreduce/

# Question 3: Understanding Total Order Sort

The key challenge in distributed computing is to break a problem into a set of sub-problems that can be performed without communicating with each other. Ideally, we should be able to define an arbirtary number of splits and still get the right result, but that is not always possible. Parallelization becomes particularly challenging when we need to make comparisons between records, for example when sorting. Total Order Sort allows us to order large datasets in a way that enables efficient retrieval of results. Before beginning this assignment, make sure you have read and understand the [Total Order Sort Notebook](https://github.com/UCB-w261/main/tree/master/HelpfulResources/TotalSortGuide/_total-sort-guide-spark2.01-JAN27-2017.ipynb). You can skip the first two MRJob sections, but the rest of section III and all of section IV are **very** important (and apply to Hadoop Streaming) so make sure to read them closely. Feel free to read the Spark sections as well but you won't be responsible for that material until later in the course. To verify your understanding, answer the following questions.

### Q3 Tasks:

* __a) short response:__ What is the difference between a partial sort, an unordered total sort, and a total order sort? From the programmer's perspective, what does total order sort allow us to do that we can't with unordered total? Why is this important with large datasets?

* __b) short response:__ Which phase of a MapReduce job is leveraged to implement Total Order Sort? Which default behaviors must be changed. Why must they be changed?

* __c) short response:__ Describe in words how to configure a Hadoop Streaming job for the custom sorting and partitioning that is required for Total Order Sort.  

* __d) short response:__ Explain why we need to use an inverse hash code function.

* __e) short response:__ Where does this function need to be located so that a Total Order Sort can be performed?

### Q3 Student Answers:

> __a)__ **Partial-Sort** - The reducer output will be lot of (partition) files, each of which contains key-value records that are sorted within each partition file based on the key. Each partition file is sorted independently of each other. Concatentating such partitions would result in an unsorted file. **Total-Sort-(Unorderd-Partitions)** - Total sort refers to an ordering of all key-value pairs based upon a specified key. This total ordering will run across all output partition files unlike the partial sort described above. One caveat here is that there is no guarantee about which chunk of output is where, therefore partition files will need to be re-stacked to generate a total ordering.  **Total-Sort-(Ordered-Partitions)** - Total sort where the partition file names are also assigned in order which guarantees that the top N results will end up in the first partition. Concatentaing these files would result in a single completely sorted file witout any further intervention. The total order sort is important for large datasets because it would be time consuming to check all of the paritions to find the subset of the results you need. 

> __b)__ The shuffle and sort phase is leveraged to implement total order sort, which is after 'map' phase and before 'reduce' phase. We need to override Hadoop default behavior, such as Hadoop performs a partial sort on mapper output keys by alphanumerically increasing order, i.e. within each partition keys are sorted. Because in total order sort, we not only want the records to be sorted within each partition, we also want records to be sorted among all partitions such that the top records end up in the first partition. Therefore, we need to include a partition function to instruct Hadoop how to partition records. 

> __c)__ Overall we need to make the following modifications:
- Modification 1: Include a partition file or partition function inside mappers. In a multi-reducer setup, an additional "partition key" is needed to instruct Hadoop how to partition records, and pass through the original (key, value) pair. 
- Modification 2: Drop internal partition key inside reducers. Because we have two keys - one for partitioning and the other for sorting, the reducer needs to drop the partition key which is used internally to aid total sort, and recover the original (key, value) pairs.
- Modification 3: Post-processing step to order partitions. Because Hadoop makes no attempt to sort partition keys – the mapping between partition key and partition index is not order-preserving. Therefore, a post-processing step is required to finish total sort. We will need to take one record from every (non-empty) partition output, sort them, and construct the appropriate ordering among partitions.

> Specifically through the following command lines:
```stream.num.map.output.key.fields
stream.map.output.field.separator
mapreduce.partition.keypartitioner.options
KeyFieldBasedComparator
keycomparator.options
partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner```


> __d)__ By default, Hadoop uses a library class `HashPartitioner` to compute the partition index for keys produced by mappers. In order to preserve partition key ordering, we will construct an "inverse hashCode function", which takes as input the desired partition index and total number of partitions, and returns a partition key. This key, when supplied to the Hadoop framework (KeyBasedPartitioner), will hash to the correct partition index.  

> __e)__ The function needs to be located in the mappers and accesible before partitioning takes place.   

# About the Data
For the main task in this portion of the homework you will train a classifier to determine whether an email represents spam or not. You will train your Naive Bayes model on a 100 record subset of the Enron Spam/Ham corpus available in the HW2 data directory (__`HW2/data/enronemail_1h.txt`__).

__Source:__   
The original data included about 93,000 emails which were made public after the company's collapse. There have been a number raw and preprocessed versions of this corpus (including those available [here](http://www.aueb.gr/users/ion/data/enron-spam/index.html) and [here](http://www.aueb.gr/users/ion/publications.html)). The subset we will use is limited to emails from 6 Enron employees and a number of spam sources. It is part of [this data set](http://www.aueb.gr/users/ion/data/enron-spam/) which was created by researchers working on personlized Bayesian spam filters. Their original publication is [available here](http://www.aueb.gr/users/ion/docs/ceas2006_paper.pdf). __`IMPORTANT!`__ _For this homework please limit your analysis to the 100 email subset which we provide. No need to download or run your analysis on any of the original datasets, those links are merely provided as context._

__Preprocessing:__  
For their work, Metsis et al. (the authors) appeared to have pre-processed the data, not only collapsing all text to lower-case, but additionally separating "words" by spaces, where "words" unfortunately include punctuation. As a concrete example, the sentence:  
>  `Hey Jon, I hope you don't get lost out there this weekend!`  

... would have been reduced by Metsis et al. to the form:  
> `hey jon , i hope you don ' t get lost out there this weekend !` 

... so we have reverted the data back toward its original state, removing spaces so that our sample sentence would now look like:
> `hey jon, i hope you don't get lost out there this weekend!`  

Thus we have at least preserved contractions and other higher-order lexical forms. However, one must be aware that this reversion is not complete, and that some object (specifically web sites) will be ill-formatted, and that all text is still lower-cased.


__Format:__   
All messages are collated to a tab-delimited format:  

>    `ID \t SPAM \t SUBJECT \t CONTENT \n`  

where:  
>    `ID = string; unique message identifier`  
    `SPAM = binary; with 1 indicating a spam message`  
    `SUBJECT = string; title of the message`  
    `CONTENT = string; content of the message`   
    
Note that either of `SUBJECT` or `CONTENT` may be "NA", and that all tab (\t) and newline (\n) characters have been removed from both of the `SUBJECT` and `CONTENT` columns.  

In [5]:
!pwd

/media/notebooks/Assignments/HW2


In [6]:
# take a look at the first 100 characters of the first 5 records (RUN THIS CELL AS IS)
!head -n 5 /media/notebooks/Assignments/HW2/{ENRON} | cut -c-100

0001.1999-12-10.farmer	0	 christmas tree farm pictures	NA
0001.1999-12-10.kaminski	0	 re: rankings	 thank you.
0001.2000-01-17.beck	0	 leadership development pilot	" sally:  what timing, ask and you shall receiv
0001.2000-06-06.lokay	0	" key dates and impact of upcoming sap implementation over the next few week
0001.2001-02-07.kitchen	0	 key hr issues going forward	 a) year end reviews-report needs generating 


In [7]:
# see how many messages/lines are in the file 
#(this number may be off by 1 if the last line doesn't end with a newline)
!wc -l /media/notebooks/Assignments/HW2/{ENRON}

100 /media/notebooks/Assignments/HW2/data/enronemail_1h.txt


In [8]:
# make the HDFS directory if it doesn't already exist
!hdfs dfs -mkdir {HDFS_DIR}

mkdir: `/user/root/HW2': File exists


In [9]:
# load the data into HDFS (RUN THIS CELL AS IS)
!hdfs dfs -copyFromLocal /media/notebooks/Assignments/HW2/{ENRON} {HDFS_DIR}/enron.txt

copyFromLocal: `/user/root/HW2/enron.txt': File exists


In [10]:
!hdfs dfs -ls {HDFS_DIR}

Found 15 items
drwxr-xr-x   - root supergroup          0 2019-09-12 16:28 /user/root/HW2/chinese-output
-rw-r--r--   1 root supergroup        118 2019-09-11 17:57 /user/root/HW2/chineseTest.txt
-rw-r--r--   1 root supergroup        106 2019-09-11 17:57 /user/root/HW2/chineseTrain.txt
drwxr-xr-x   - root supergroup          0 2019-09-10 19:20 /user/root/HW2/eda-output
drwxr-xr-x   - root supergroup          0 2019-09-10 19:01 /user/root/HW2/eda-sort-output
drwxr-xr-x   - root supergroup          0 2019-09-14 01:53 /user/root/HW2/enron-model
-rw-r--r--   1 root supergroup     204559 2019-09-10 16:50 /user/root/HW2/enron.txt
-rw-r--r--   1 root supergroup      41493 2019-09-13 00:38 /user/root/HW2/enron_test.txt
-rw-r--r--   1 root supergroup     163066 2019-09-13 00:38 /user/root/HW2/enron_train.txt
drwxr-xr-x   - root supergroup          0 2019-09-12 20:16 /user/root/HW2/nbModel-output
drwxr-xr-x   - root supergroup          0 2019-09-12 21:19 /user/root/HW2/nbModelSmooth-output
drwxr-x

# Question 4:  Enron Ham/Spam EDA.
Before building our classifier, lets get aquainted with our data. In particular, we're interested in which words occur more in spam emails than in real emails. In this question you'll implement two Hadoop MapReduce jobs to count and sort word occurrences by document class. You'll also learn about two new Hadoop streaming parameters that will allow you to control how the records output of your mappers are partitioned for reducing on separate nodes. 

__`IMPORTANT NOTE:`__ For this question and all subsequent items, you should include both the subject and the body of the email in your analysis (i.e. concatetate them to get the 'text' of the document).

### Q4 Tasks:
* __a) code:__ Complete the missing components of the code in __`EnronEDA/mapper.py`__ and __`EnronEDA/reducer.py`__ to create a Hadoop MapReduce job that counts how many times each word in the corpus occurs in an email for each class. Pay close attention to the data format specified in the docstrings of these scripts _-- there are a number of ways to accomplish this task, we've chosen this format to help illustrate a technique in `part e`_. Run the provided unit tests to confirm that your code works as expected then run the provided Hadoop streaming command to apply your analysis to the Enron data.


* __b) code + short response:__ How many times does the word "__assistance__" occur in each class? (`HINT:` Use a `grep` command to read from the results file you generated in '`a`' and then report the answer in the space provided.)


* __c) short response:__ Would it have been possible to add some sorting parameters to the Hadoop streaming command that would cause our `part a` results to be sorted by count? Briefly explain why or why not.


* __d) code + short response:__ Write a second Hadoop MapReduce job to sort the output of `part a` first by class and then by count. Run your job and save the results to a local file. Then describe in words how you would go about printing the top 10 words in each class given this sorted output. (`HINT 1:` _remember that you can simply pass the `part a` output directory to the input field of this job; `HINT 2:` since this task is just reodering the records from `part a` we don't need to write a mapper or reducer, just use `/bin/cat` for both_)


* __e) code:__ A more efficient alternative to '`grep`-ing' for the top 10 words in each class would be to use the Hadoop framework to separate records from each class into its own partition so that we can just read the top lines in each. Edit your job from ` part d` to specify 2 reduce tasks and to tell Hadoop to partition based on the second field (which indicates spam/ham in our data). Your code should maintain the secondary sort -- that is each partition should list words from most to least frequent.

### Q4 Student Answers:
> __b)__ The word "**assistance**" occurs 8 times in Class 1 (spam), and 2 times in Class 0 (ham).   

> __c)__ No, we can’t sort on counts in the original job because Hadoop’s sorting occurs in the phase between the mapper and reducer but our counts aren’t tallied up until after the reducer.

> __d)__ I would partition the results based on class, i.e. class 0 records go to one partition file, and class 1 records go to the other partition file. From there, we get the top 10 words from each partition file. We could also grep for the class marker that is listed in the second field.

In [11]:
# part a - do your work in the provided scripts then RUN THIS CELL AS IS
!chmod a+x EnronEDA/mapper.py
!chmod a+x EnronEDA/reducer.py

In [12]:
# part a - unit test EnronEDA/mapper.py (RUN THIS CELL AS IS)
!echo -e "d1	1	title	body\nd2	0	title	body" | EnronEDA/mapper.py

title	1	1
body	1	1
title	0	1
body	0	1


In [13]:
# part a - unit test EnronEDA/reducer.py (RUN THIS CELL AS IS)
!echo -e "one	1	1\none	0	1\none	0	1\ntwo	0	1" | EnronEDA/reducer.py

one	1	1
one	0	2
two	1	0
two	0	1


In [14]:
# part a - clear output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-output

Deleted /user/root/HW2/eda-output


In [15]:
# part a - Hadoop streaming job (RUN THIS CELL AS IS)
!hadoop jar {JAR_FILE} \
  -files EnronEDA/reducer.py,EnronEDA/mapper.py \
  -mapper mapper.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/enron.txt \
  -output {HDFS_DIR}/eda-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob7859609207462294535.jar tmpDir=null
19/09/16 16:13:18 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/16 16:13:18 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/16 16:13:19 INFO mapred.FileInputFormat: Total input paths to process : 1
19/09/16 16:13:19 INFO mapreduce.JobSubmitter: number of splits:2
19/09/16 16:13:19 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1567779136510_0041
19/09/16 16:13:20 INFO impl.YarnClientImpl: Submitted application application_1567779136510_0041
19/09/16 16:13:20 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1567779136510_0041/
19/09/16 16:13:20 INFO mapreduce.Job: Running job: job_1567779136510_0041
19/09/16 16:13:28 INFO mapreduce.Job: Job job_1567779136510_0041 running in uber mode : false
19/09/16 16:13:28 INFO mapreduce.Job:  map 0% reduce 0%
19/09

In [16]:
# part a - retrieve results from HDFS & copy them into a local file (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/eda-output/part-0000* > EnronEDA/results.txt

In [17]:
# part b - write your grep command here
!grep assistance EnronEDA/results.txt

assistance	1	8
assistance	0	2


In [18]:
# part d/e - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-sort-output

Deleted /user/root/HW2/eda-sort-output


In [19]:
# part d/e - write your Hadoop streaming job here
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keypartitioner.options="-k2,2" \
  -D mapreduce.partition.keycomparator.options="-k3,3nr" \
  -mapper /bin/cat \
  -reducer /bin/cat \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -input {HDFS_DIR}/eda-output/part-0000* \
  -output {HDFS_DIR}/eda-sort-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob722682705600919387.jar tmpDir=null
19/09/16 16:21:05 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/16 16:21:05 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/16 16:21:06 INFO mapred.FileInputFormat: Total input paths to process : 2
19/09/16 16:21:06 INFO mapreduce.JobSubmitter: number of splits:2
19/09/16 16:21:06 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1567779136510_0042
19/09/16 16:21:06 INFO impl.YarnClientImpl: Submitted application application_1567779136510_0042
19/09/16 16:21:07 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1567779136510_0042/
19/09/16 16:21:07 INFO mapreduce.Job: Running job: job_1567779136510_0042
19/09/16 16:21:16 INFO mapreduce.Job: Job job_1567779136510_0042 running in uber mode : false
19/09/16 16:21:16 INFO mapreduce.Job:  map 0% reduce 0%
19/09/

In [20]:
# part e - view the top 10 records from each partition (RUN THIS CELL AS IS)
for idx in range(2):
    print(f"\n===== part-0000{idx}=====\n")
    !hdfs dfs -cat {HDFS_DIR}/eda-sort-output/part-0000{idx} | head


===== part-00000=====

the	0	549	
to	0	398	
ect	0	382	
and	0	278	
of	0	230	
hou	0	206	
a	0	196	
in	0	182	
for	0	170	
on	0	135	
cat: Unable to write to output stream.

===== part-00001=====

the	1	698	
to	1	566	
and	1	392	
your	1	357	
a	1	347	
you	1	345	
of	1	336	
in	1	236	
for	1	204	
com	1	153	
cat: Unable to write to output stream.


__Expected output:__
<table>
<th>part-00000:</th>
<th>part-00001:</th>
<tr><td><pre>
the	0	549	
to	0	398	
ect	0	382	
and	0	278	
of	0	230	
hou	0	206	
a	0	196	
in	0	182	
for	0	170	
on	0	135
</pre></td>
<td><pre>
the	1	698	
to	1	566	
and	1	392	
your	1	357	
a	1	347	
you	1	345	
of	1	336	
in	1	236	
for	1	204	
com	1	153
</pre></td></tr>
</table>

# Question 5: Counters and Combiners.
Tuning the number of mappers & reducers is helpful to optimize very large distributed computations. Doing so successfully requires a thorough understanding of the data size at each stage of the job. As you learned in the week3 live session, counters are an invaluable resource for understanding this kind of detail. In this question, we will take the EDA performed in Question 4 as an opportunity to illustrate some related concepts.

### Q5 Tasks:
* __a) short response:__ Read the Hadoop output from your job in Question 4a to report how many records are emitted by the mappers and how many records are received be the reducers. In the context of word counting what does this number represent practically?

* __b) code:__ Note that we wrote the reducer in question 4a such that the input and output record format is identical. This makes it easy to use the same reducer script as a combiner. In the space provided below, write the Hadoop Streaming command to re-run your job from question 4a with this combining added.

* __c) short response__: Report the number of records emitted by your mappers in part b and the number of records received by your reducers. Compare your results here to what you saw in part a. Explain.

* __d) short response__: Describe a scenario where using a combiner would _NOT_ improve the efficiency of the shuffle stage. Explain. [__`BONUS:`__ how does increasing the number of mappers affect the usefulness of a combiner?]

### Q5 Student Answers:
> __a)__ Mappers emitted 31490 records; reducers received 31490 records. This number represents the total words in the subject & content data of 100 Enron emails. 

> __c)__ Mappers emitted 31490 records; reducers received 13096 records (which is the number of records emitted by combiners). Because we use combiners at the mappers, the output records emitted by each mapper have been locally aggregated before being sent over to reducers. That's why we see the number of records received by reducers is less than part b. 

> __d)__ The output records emitted by each mapper only contain distinct keys, no duplicate keys. Increasing the number of mappers would limit or decrease the usefulnees of a combiner. This is because the more data are chunked up and sent to more mappers, the less chance they'll be duplicate records to combine.   

In [21]:
# part b - clear output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-output

Deleted /user/root/HW2/eda-output


In [22]:
# part b - write your Hadoop streaming job here
!hadoop jar {JAR_FILE} \
  -files EnronEDA/reducer.py,EnronEDA/mapper.py \
  -mapper mapper.py \
  -combiner reducer.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/enron.txt \
  -output {HDFS_DIR}/eda-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob8261785998608777901.jar tmpDir=null
19/09/16 16:44:00 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/16 16:44:00 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/16 16:44:01 INFO mapred.FileInputFormat: Total input paths to process : 1
19/09/16 16:44:01 INFO mapreduce.JobSubmitter: number of splits:2
19/09/16 16:44:01 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1567779136510_0043
19/09/16 16:44:01 INFO impl.YarnClientImpl: Submitted application application_1567779136510_0043
19/09/16 16:44:01 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1567779136510_0043/
19/09/16 16:44:01 INFO mapreduce.Job: Running job: job_1567779136510_0043
19/09/16 16:44:10 INFO mapreduce.Job: Job job_1567779136510_0043 running in uber mode : false
19/09/16 16:44:10 INFO mapreduce.Job:  map 0% reduce 0%
19/09

# Question 6: Document Classification Task Overview.
The week 2 assigned reading from Chapter 13 of _Introduction to Information Retrieval_ by Manning, Raghavan and Schutze provides a thorough introduction to the document classification task and the math behind Naive Bayes. In this question we'll use the example from Table 13.1 (reproduced below) to 'train' an unsmoothed Multinomial Naive Bayes model and classify a test document by hand.

<table>
<th>DocID</th>
<th>Class</th>
<th>Subject</th>
<th>Body</th>
<tr><td>Doc1</td><td>1</td><td></td><td>Chinese Beijing Chinese</td></tr>
<tr><td>Doc2</td><td>1</td><td></td><td>Chinese Chinese Shanghai</td></tr>
<tr><td>Doc3</td><td>1</td><td></td><td>Chinese Macao</td></tr>
<tr><td>Doc4</td><td>0</td><td></td><td>Tokyo Japan Chinese</td></tr>
</table>

### Q6 Tasks:
* __a) short response:__ Equation 13.3 in Manning, Raghavan and Shutze shows how a Multinomial Naive Bayes model classifies a document. It predicts the class, $c$, for which the estimated conditional probability of the class given the document's contents,  $\hat{P}(c|d)$, is greatest. In this equation what two pieces of information are required to calculate  $\hat{P}(c|d)$? Your answer should include both mathematical notatation and verbal explanation.


* __b) short response:__ The Enron data includes two classes of documents: `spam` and `ham` (they're actually labeled `1` and `0`). In plain English, explain what  $\hat{P}(c)$ and   $\hat{P}(t_{k} | c)$ mean in the context of this data. How will we would estimate these values from a training corpus? How many passes over the data would we need to make to retrieve this information for all classes and all words?


* __c) hand calculations:__ Above we've reproduced the document classification example from the textbook (we added an empty subject field to mimic the Enron data format). Remember that the classes in this "Chinese Example" are `1` (about China) and `0` (not about China). Calculate the class priors and the conditional probabilities for an __unsmoothed__ Multinomial Naive Bayes model trained on this data. Show the calculations that lead to your result using markdown and $\LaTeX$ in the space provided or by embedding an image of your hand written work. [`NOTE:` _Your results should NOT match those in the text -- they are training a model with +1 smoothing you are training a model without smoothing_]


* __d) hand calculations:__ Use the model you trained to classify the following test document: `Chinese Chinese Chinese Tokyo Japan`. Show the calculations that lead to your result using markdown and   $\LaTeX$ in the space provided or by embedding an image of your hand written work.


* __e) short response:__ Compare the classification you get from this unsmoothed model in `d`/`e` to the results in the textbook's "Example 1" which reflects a model with Laplace plus 1 smoothing. How does smoothing affect our inference?

### Q6 Student Answers:
> __a)__ $\hat{P}(c|d) = \hat{P}(c)\Pi \hat{P}(t_{k} | c)$ The two pieces of information are:
- $\hat{P}(t_{k} | c)$: the estimated conditional probability of term $t_{k}$ occurring in a document of class c. It's a measure of how much evidence $t_{k}$ contributes that c is the correct class.
- $\hat{P}(c)$: the estimated prior probability of a document occurring in class c.

> __b)__ For Enron's case:
- $\hat{P}(t_{k} | c)$: the estimated conditional probability of a word $t_{k}$ occurring in a document of class Spam or Ham. For example the word **a**, we need to count the number of documents the word 'a' appears for a given class, i.e. Spam, and then divided by the count of Spam emails, or, $\hat{P}(a | Spam) = \frac{count(a | Spam)}{count(Spam)}$.
- $\hat{P}(c)$: the estimated prior probability of class Spam or Ham. $\hat{P}(Spam) + \hat{P}(Ham) = 1$. We count the number of Spam emails, and the number of Ham emails. $\hat{P}(Spam) = \frac{count(Spam)}{count(Spam + Ham)}$

> __c)__ The calculations are shown as below:
- Note that there are 3 documents labeled as "China", and 1 document labeled as "Not China", hence $count(China)=3$, and $count(NotChina)=1$. Therefore, the priors: 
    - $\hat{P}(China) = \frac{1 + 1 + 1}{4} = 0.75$ 
    - $\hat{P}(NotChina) = \frac{1}{4} = 0.25$
- Distinct evidences (or features): Chinese, Beijing, Shanghai, Macao, Tokyo, Japan. Their counts for respective classes are shown below:

<table>
<th>Class</th>
<th>Chinese</th>
<th>Beijing</th>
<th>Shanghai</th>
<th>Macao</th>
<th>Tokyo</th>
<th>Japan</th>
<th>Total</th>
<tr><td>1 - China</td><td>5 (i)</td><td>1</td><td>1</td><td>1</td><td>0</td><td>0</td><td>8</td></tr>
<tr><td>0 - NotChina</td><td>1</td><td>0</td><td>0</td><td>0</td><td>1</td><td>1</td><td>3</td></tr>
</table>

>* (i): The word "Chinese" appear 5 times in the documents that are labeled as "1 - China". Because we are to construct a Multinomial Naive Bayes model, we need to count the occurrences of a word.  

>- Their respective conditional probabilities for each class are shown as below, :
    - $\hat{P}(Chinese | China) = \frac{count(Chinese | China)}{total(China)} = \frac{5}{8}$
    - $\hat{P}(Chinese | NotChina) = \frac{count(Chinese | NotChina)}{total(NotChina)} = \frac{1}{3}$
    - $\hat{P}(Beijing | China) = \frac{1}{8}$
    - $\hat{P}(Beijing | NotChina) = \frac{0}{3} = 0$
    - $\hat{P}(Shanghai | China) = \frac{1}{8}$
    - $\hat{P}(Shanghai | NotChina) = \frac{0}{3} = 0$
    - $\hat{P}(Macao | China) = \frac{1}{8}$
    - $\hat{P}(Macao | NotChina) = \frac{0}{3} = 0$
    - $\hat{P}(Tokyo | China) = \frac{0}{8} = 0$
    - $\hat{P}(Tokyo | NotChina) = \frac{1}{3}$
    - $\hat{P}(Japan | China) = \frac{0}{8} = 0$
    - $\hat{P}(Japan | NotChina) = \frac{1}{3}$

> __d)__ Doc5 = 'Chinese Chinese Chinese Tokyo Japan'
$$
\begin{aligned}
\hat{P}(China | Doc5) &= \hat{P}(China) \cdot \left[ \hat{P}(Chinese | China) \right]^{3} \cdot \hat{P}(Tokyo | China) \cdot \hat{P}(Japan | China) \\
&= \frac{3}{4} \cdot \left( \frac{5}{8} \right)^{3} \cdot 0 \cdot 0 \\
&= 0
\end{aligned}
$$
$$
\begin{aligned}
\hat{P}(NotChina | Doc5) &= \hat{P}(NotChina) \cdot \left[ \hat{P}(Chinese | NotChina) \right]^{3} \cdot \hat{P}(Tokyo | NotChina) \cdot \hat{P}(Japan | NotChina) \\
&= \frac{1}{4} \cdot \left( \frac{1}{3} \right)^{3} \cdot \frac{1}{3} \cdot \frac{1}{3} \\
&= \frac{1}{972} 
\end{aligned}
$$

> Based on the calculation, the predicted class is Not China. 

> __e)__ Smoothing avoid zero count scenario, such as a word that has never been seen in training data, or situation that we run into as in part __c__ and __d__, in which the zero term makes the whole probability zero. Smoothing is also like adding noise to training data to improve generalization of our model. As we see in part __d__, smoothing could in fact change our inference results as compared with unsmoothed model. 


In [ ]:
# part d/e - if you didn't write out your calcuations above, embed a picture of them here:
from IPython.display import Image
Image(filename="path-to-hand-calulations-image.png")

# Question 7: Naive Bayes Inference.
In the next two questions you'll write code to parallelize the Naive Bayes calculations that you performed above. We'll do this in two phases: one MapReduce job to perform training and a second MapReduce to perform inference. While in practice we'd need to train a model before we can use it to classify documents, for learning purposes we're going to develop our code in the opposite order. By first focusing on the pieces of information/format we need to perform the classification (inference) task you should find it easier to develop a solid implementation for training phase when you get to question 8 below. In both of these questions we'll continue to use the Chinese example corpus from the textbook to help us test our MapReduce code as we develop it. Below we've reproduced the corpus, test set and model in text format that matches the Enron data.

### Q7 Tasks:
* __a) short response:__ run the provided cells to create the example files and load them in to HDFS. Then take a closer look at __`NBmodel.txt`__. This text file represents a Naive Bayes model trained (with Laplace +1 smoothing) on the example corpus. What are the 'keys' and 'values' in this file? Which record means something slightly different than the rest? The value field of each record includes two numbers which will be helpful for debugging but which we don't actually need to perform inference -- what are they? [`HINT`: _This file represents the model from Example 13.1 in the textbook, if you're having trouble getting oriented try comparing our file to the numbers in that example._]


* __b) short response:__ When performing Naive Bayes in practice instead of multiplying the probabilities (as in equation 13.3) we add their logs (as in equation 13.4). Why do we choose to work with log probabilities? If we had an unsmoothed model, what potential error could arise from this transformation?


* __c) short response:__ Documents 6 and 8 in the test set include a word that did not appear in the training corpus (and as a result does not appear in the model). What should we do at inference time when we need a class conditional probability for this word?


* __d) short response:__ The goal of our MapReduce job is to stream over the test set and classify each document by peforming the calculation from equation 13.4. To do this we'll load the model file (which contains the probabilities for equation 13.4) into memory on the nodes where we do our mapping. This is called an in-memory join. Does loading a model 'state' like this depart from the functional programming principles? Explain why or why not. From a scability perspective when would this kind of memory use be justified? when would it be unwise?


* __e) code:__ Complete the code in __`NaiveBayes/classify_mapper.py`__. Read the docstring carefully to understand how this script should work and the format it should return. Run the provided unit tests to confirm that your script works as expected then write a Hadoop streaming job to classify the Chinese example test set. [`HINT 1:` _you shouldn't need a reducer for this one._ `HINT 2:` _Don't forget to add the model file to the_ `-files` _parameter in your Hadoop streaming job so that it gets shipped to the mapper nodes where it will be accessed by your script._]


* __f) short response:__ In our test example and in the Enron data set we have fairly short documents. Since these fit fine in memory on a mapper node we didn't need a reducer and could just do all of our calculations in the mapper. However with much longer documents (eg. books) we might want a higher level of parallelization -- for example we might want to process parts of a document on different nodes. In this hypothetical scenario how would our algorithm design change? What could the mappers still do? What key-value structure would they emit? What would the reducers have to do as a last step?

### Q7 Student Answers:
> __a)__ See response below:
- __Keys__ are the words appear in the training data corpus, specifically in our trained model, they are, beijing, chinese, tokyo, shanghai, ClassPriors, japan, macao; 
- __Values__ are the following numbers, count in class 0, count in class 1, P(w|class0), P(w|class1).
- __ClassPriors__ record means slightly different than other records, as it is not a word in the corpus, but rather a summary line for the two classes to show their total counts and prior probabilities. 
- The first two numeric fields (count in class 0, count in class 1) are not needed to perform inference but helpful for debugging. They are the counts for each word appearing in the two classes, NotChina and China respectively. 

> __b)__ We choose to use log probabilities because:
- First of all, we do not care about absolute probability, but rather a relative one for comparison;
- With many conditional probabilities are multiplied, it could result in a floating point underflow.
- By adding logarithms of probabilities instead of multiplying probabilities, the class with the highest log probability score is still the most probable. 
- If we use unsmoothed model, we could run into $log(0)$ which is undefined, and not a real number mathematically. 

> __c)__ We should add in Laplace smoothing, where we should set pseudocount $\alpha > 0$. We could either assign the same conditional probability to each class or we could simply disregard that word. 

> __d)__ It slightly departs from functional programming principles. However it is ok since we’re not updating that state at all after it gets loaded on each mapper node so there’s no risk of a race condition. So the model file essentially is like a function where the output value of our model depends only on the arguments that are passed to our model. In addition, we do not share any information that needs to be updated across multiple nodes. From a scalability perspective, the use of "in memory join" is justified when the input/output of the join is small enough to fit the memory. Otherwise, we would need to use different approaches. 

> __e)__ Complete the coding portion of this question before answering 'f'.

> __f)__ If the documents get long enough that they no longer fit the memory, we would need to process parts of each document in parallel. Then the mapper should no longer be responsible for adding the log of the class priors (because that would end up getting added multiple times-- once for each mapper that encounters a piece of the same document). Our new mapper could still perform the conditional probability look-ups (emitting `DocID \t log(P(word|class0)),log(P(word|class1)`). Then the reducers would add the log probabilities of all the words associated with each document (from all the mappers that processed a piece of that document) and then add the ClassPriors to make a final inference.


Run these cells to create the example corpus and model.

In [23]:
%%writefile NaiveBayes/chineseTrain.txt
D1	1		Chinese Beijing Chinese
D2	1		Chinese Chinese Shanghai
D3	1		Chinese Macao
D4	0		Tokyo Japan Chinese

Overwriting NaiveBayes/chineseTrain.txt


In [24]:
%%writefile NaiveBayes/chineseTest.txt
D5	1		Chinese Chinese Chinese Tokyo Japan
D6	1		Beijing Shanghai Trade
D7	0		Japan Macao Tokyo
D8	0		Tokyo Japan Trade

Overwriting NaiveBayes/chineseTest.txt


In [25]:
%%writefile NBmodel.txt
beijing	0.0,1.0,0.111111111111,0.142857142857
chinese	1.0,5.0,0.222222222222,0.428571428571
tokyo	1.0,0.0,0.222222222222,0.0714285714286
shanghai	0.0,1.0,0.111111111111,0.142857142857
ClassPriors	1.0,3.0,0.25,0.75
japan	1.0,0.0,0.222222222222,0.0714285714286
macao	0.0,1.0,0.111111111111,0.142857142857

Overwriting NBmodel.txt


In [26]:
# load the data files into HDFS
!hdfs dfs -copyFromLocal NaiveBayes/chineseTrain.txt {HDFS_DIR}
!hdfs dfs -copyFromLocal NaiveBayes/chineseTest.txt {HDFS_DIR}

copyFromLocal: `/user/root/HW2/chineseTrain.txt': File exists
copyFromLocal: `/user/root/HW2/chineseTest.txt': File exists


Your work for `part e` starts here:

In [27]:
# part e - do your work in NaiveBayes/classify_mapper.py first, then run this cell.
!chmod a+x NaiveBayes/classify_mapper.py

In [28]:
# part e - unit test NaiveBayes/classify_mapper.py (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py | column -t

d5  1  -8.90668134500626   -8.10769031284611   1
d6  1  -5.780743515794329  -4.179502370564408  1
d7  0  -6.591673732011658  -7.511706880737812  0
d8  0  -4.394449154674438  -5.565796731681498  0


In [29]:
# part e - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/chinese-output

Deleted /user/root/HW2/chinese-output


In [30]:
# part e - write your Hadooop streaming job here
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/classify_mapper.py,NBmodel.txt \
  -mapper classify_mapper.py \
  -reducer /bin/cat \
  -input {HDFS_DIR}/chineseTest.txt \
  -output {HDFS_DIR}/chinese-output \
  -numReduceTasks 1 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob3249677114930579200.jar tmpDir=null
19/09/16 17:36:45 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/16 17:36:45 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/16 17:36:46 INFO mapred.FileInputFormat: Total input paths to process : 1
19/09/16 17:36:46 INFO mapreduce.JobSubmitter: number of splits:2
19/09/16 17:36:46 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1567779136510_0044
19/09/16 17:36:47 INFO impl.YarnClientImpl: Submitted application application_1567779136510_0044
19/09/16 17:36:47 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1567779136510_0044/
19/09/16 17:36:47 INFO mapreduce.Job: Running job: job_1567779136510_0044
19/09/16 17:36:55 INFO mapreduce.Job: Job job_1567779136510_0044 running in uber mode : false
19/09/16 17:36:55 INFO mapreduce.Job:  map 0% reduce 0%
19/09

In [31]:
# part e - retrieve test set results from HDFS (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/chinese-output/part-000* > NaiveBayes/chineseResults.txt

In [32]:
# part e - take a look (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseResults.txt | column -t

d5  1  -8.90668134500626   -8.10769031284611   1
d6  1  -5.780743515794329  -4.179502370564408  1
d7  0  -6.591673732011658  -7.511706880737812  0
d8  0  -4.394449154674438  -5.565796731681498  0


<table>
<th> Expected output for the test set:</th>
<tr align=Left><td><pre>
d5	1	-8.90668134	-8.10769031	1
d6	1	-5.78074351	-4.17950237	1
d7	0	-6.59167373	-7.51170688	0
d8	0	-4.39444915	-5.56579673	0
</pre></td><tr>
</table>

# Question 8: Naive Bayes Training.
In Question 7 we used a model that we had trained by hand. Next we'll develop the code to do that same training in parallel, making it suitable for use with larger corpora (like the Enron emails). The end result of the MapReduce job you write in this question should be a model text file that looks just like the example (`NBmodel.txt`) that we created by hand above.

To refresh your memory about the training process take a look at  `6a` and `6b` where you described the pieces of information you'll need to collect in order to encode a Multinomial Naive Bayes model. We now want to retrieve those pieces of information while streaming over a corpus. The bulk of the task will be very similar to the word counting excercises you've already done but you may want to consider a slightly different key-value record structure to efficiently tally counts for each class. 

The most challenging (interesting?) design question will be how to retrieve the totals (# of documents and # of words in documents for each class). Of course, counting these numbers is easy. The hard part is the timing: you'll need to make sure you have the counts totalled up _before_ you start estimating the class conditional probabilities for each word. It would be best (i.e. most scalable) if we could find a way to do this tallying without storing the whole vocabulary in memory... Use an appropriate MapReduce design pattern to implement this efficiently! 


### Q8 Tasks:
* __a) make a plan:__  Fill in the docstrings for __`NaiveBayes/train_mapper.py`__ and __`NaiveBayes/train_reducer.py`__ to appropriately reflect the format that each script will input/output. [`HINT:` _the input files_ (`enronemail_1h.txt` & `chineseTrain.txt`) _have a prespecified format and your output file should match_ `NBmodel.txt` _so you really only have to decide on an internal format for Hadoop_].


* __b) implement it:__ Complete the code in __`NaiveBayes/train_mapper.py`__ and __`NaiveBayes/train_reducer.py`__ so that together they train a Multinomial Naive Bayes model __with no smoothing__. Make sure your end result is formatted correctly (see note above). Test your scripts independently and together (using `chineseTrain.txt` or test input of your own devising). When you are satisfied with your Python code design and run a Hadoop streaming command to run your job in parallel on the __chineseTrain.txt__. Confirm that your trained model matches your hand calculations from Question 5.


* __c) short response:__ We saw in Question 6 that adding Laplace +1 smoothing makes our classifications less sensitve to rare words. However implementing this technique requires access to one additional piece of information that we had not previously used in our Naive Bayes training. What is that extra piece of information? [`HINT:` see equation 13.7 in Manning, Raghavan and Schutze].


* __d) short response:__ There are a couple of approaches that we could take to handle the extra piece of information you identified in `c`: 1) if we knew this extra information beforehand, we could provide it to our reducer as a configurable parameter for the vocab size dynamically (_where would we get it in the first place?_). Or 2) we could compute it in the reducer without storing any bulky information in memory but then we'd need some postprocessing or a second MapReduce job to complete the calculation (_why?_). Breifly explain what is non-ideal about each of these options. 


* __e) code + short response:__ Choose one of the 2 options above. State your choice & reasoning in the space below then use that strategy to complete the code in __`NaiveBayes/train_reducer_smooth.py`__. Test this alternate reducer then write and run a Hadoop streaming job to train an MNB model with smoothing on the Chinese example. Your results should match the model that we provided for you above (and the calculations in the textbook example). [`HINT:` _don't start from scratch with this one -- you can just copy over your reducer code from part `b` and make the needed modifications_].


__IMPORTANT NOTE:__ For full credit on this question, your code must work with multiple reducers. [`HINT:`_You will need to implement custom partitioning - [Total Order Sort Notebook](https://github.com/UCB-w261/main/tree/master/HelpfulResources/TotalSortGuide/_total-sort-guide-spark2.01-JAN27-2017.ipynb)]


### Q8 Student Answers:
> __c)__ The vocabulary size (number of distinct words) of our training data. 

> __d)__ Option (1) We would get the extra information by performing EDA on our train data; Option (2) To avoid situation where there could be potentially large volume of partial counts for each word in memory.
- Option (1): it is not ideal in that everytime we run on a different set of training data, we need to redo our EDA and get the vocabulary size;
- Option (2): it takes longer time to run compared to Option 1. 

> __e)__ I choose option (1) - perform EDA on train data to get our vocabulary size, because it is easier and faster. 


In [51]:
# part a - do your work in train_mapper.py and train_reducer.py then RUN THIS CELL AS IS
!chmod a+x NaiveBayes/train_mapper.py
!chmod a+x NaiveBayes/train_reducer.py
!echo "=========== MAPPER DOCSTRING ============"
!head -n 8 NaiveBayes/train_mapper.py | tail -n 6
!echo "=========== REDUCER DOCSTRING ============"
!head -n 8 NaiveBayes/train_reducer.py | tail -n 6

=========== MAPPER DOCSTRING ============
Mapper reads in text documents and emits word counts by class.
INPUT:                                                    
    DocID \t true_class \t subject \t body                
OUTPUT:                                                   
    word \t class0_partialCount,class1_partialCount       
    
=========== REDUCER DOCSTRING ============
Reducer aggregates word counts by class and emits frequencies.
INPUT:
    word \t class0_partialCount,class1_partialCount
OUTPUT:
    word \t ham_count,spam_count,P(ham|word),P(spam|word)
    


__`part b starts here`:__ MNB _without_ Smoothing (training on Chinese Example Corpus).

In [53]:
# part b - write a unit test for your mapper here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py

chinese	0,1
beijing	0,1
chinese	0,1
chinese	0,1
chinese	0,1
shanghai	0,1
chinese	0,1
macao	0,1
tokyo	1,0
japan	1,0
chinese	1,0
*docTotals	1,3
*wordTotals	3,8


In [54]:
# part b - write a unit test for your reducer here
!echo -e "*docTotals\t1,3\n*wordTotals\t3,8\njapan\t1,0\nchinese\t0,1\nchinese\t0,1\nchinese\t0,1\nchinese\t0,1\nchinese\t0,1\nbeijing\t0,1\nshanghai\t0,1\nmacao\t0,1\ntokyo\t1,0" | NaiveBayes/train_reducer.py

japan	1.0,0.0,0.3333333333333333,0.0
chinese	0.0,5.0,0.0,0.625
beijing	0.0,1.0,0.0,0.125
shanghai	0.0,1.0,0.0,0.125
macao	0.0,1.0,0.0,0.125
tokyo	1.0,0.0,0.3333333333333333,0.0
ClassPriors	1.0,3.0,0.25,0.75


In [55]:
# part b - write a systems test for your mapper + reducer together here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py | sort | NaiveBayes/train_reducer.py

beijing	0.0,1.0,0.0,0.125
chinese	1.0,5.0,0.3333333333333333,0.625
japan	1.0,0.0,0.3333333333333333,0.0
macao	0.0,1.0,0.0,0.125
shanghai	0.0,1.0,0.0,0.125
tokyo	1.0,0.0,0.3333333333333333,0.0
ClassPriors	1.0,3.0,0.25,0.75


In [56]:
# part b - clear (and name) an output directory in HDFS for your unsmoothed chinese NB model
!hdfs dfs -rm -r {HDFS_DIR}/nbModel-output

Deleted /user/root/HW2/nbModel-output


In [57]:
# part b - write your hadoop streaming job
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer.py \
  -mapper train_mapper.py \
  -reducer train_reducer.py \
  -input {HDFS_DIR}/chineseTrain.txt \
  -output {HDFS_DIR}/nbModel-output \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob8649631958262128303.jar tmpDir=null
19/09/16 18:24:33 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/16 18:24:33 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/16 18:24:34 INFO mapred.FileInputFormat: Total input paths to process : 1
19/09/16 18:24:34 INFO mapreduce.JobSubmitter: number of splits:2
19/09/16 18:24:34 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1567779136510_0047
19/09/16 18:24:35 INFO impl.YarnClientImpl: Submitted application application_1567779136510_0047
19/09/16 18:24:35 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1567779136510_0047/
19/09/16 18:24:35 INFO mapreduce.Job: Running job: job_1567779136510_0047
19/09/16 18:24:44 INFO mapreduce.Job: Job job_1567779136510_0047 running in uber mode : false
19/09/16 18:24:44 INFO mapreduce.Job:  map 0% reduce 0%
19/09

In [58]:
# part b - extract your results (i.e. model) to a local file
!hdfs dfs -cat {HDFS_DIR}/nbModel-output/part-000* > NaiveBayes/UnsmoothNBModel.txt

In [59]:
# part b - print your model so that we can confirm that it matches expected results
!cat NaiveBayes/UnsmoothNBModel.txt | column -t

beijing      0.0,1.0,0.0,0.125
chinese      1.0,5.0,0.3333333333333333,0.625
japan        1.0,0.0,0.3333333333333333,0.0
macao        0.0,1.0,0.0,0.125
shanghai     0.0,1.0,0.0,0.125
tokyo        1.0,0.0,0.3333333333333333,0.0
ClassPriors  1.0,3.0,0.25,0.75


In [60]:
!cat NaiveBayes/chineseTrain.txt

D1	1		Chinese Beijing Chinese
D2	1		Chinese Chinese Shanghai
D3	1		Chinese Macao
D4	0		Tokyo Japan Chinese

__`part e starts here`:__ MNB _with_ Smoothing (training on Chinese Example Corpus).

In [61]:
# part e - write a unit test for your NEW reducer here
!chmod a+x NaiveBayes/train_reducer_smooth.py 
# Please note that originally provided codes have been modified 
# to make sure its output match with the input format of model_sort_mapper 
!echo -e "*docTotals\t1,3\n*wordTotals\t3,8\njapan\t1,0\nchinese\t0,1\nchinese\t0,1\nchinese\t0,1\nchinese\t0,1\nchinese\t0,1\nbeijing\t0,1\nshanghai\t0,1\nmacao\t0,1\ntokyo\t1,0" | NaiveBayes/train_reducer_smooth.py

japan	1.0,0.0,0.2222222222222222,0.07142857142857142
chinese	0.0,5.0,0.1111111111111111,0.42857142857142855
beijing	0.0,1.0,0.1111111111111111,0.14285714285714285
shanghai	0.0,1.0,0.1111111111111111,0.14285714285714285
macao	0.0,1.0,0.1111111111111111,0.14285714285714285
tokyo	1.0,0.0,0.2222222222222222,0.07142857142857142
ClassPriors	1.0,3.0,0.25,0.75


In [62]:
# part e - write a systems test for your mapper + reducer together here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py | sort | NaiveBayes/train_reducer_smooth.py

beijing	0.0,1.0,0.1111111111111111,0.14285714285714285
chinese	1.0,5.0,0.2222222222222222,0.42857142857142855
japan	1.0,0.0,0.2222222222222222,0.07142857142857142
macao	0.0,1.0,0.1111111111111111,0.14285714285714285
shanghai	0.0,1.0,0.1111111111111111,0.14285714285714285
tokyo	1.0,0.0,0.2222222222222222,0.07142857142857142
ClassPriors	1.0,3.0,0.25,0.75


In [63]:
# part e - clear (and name) an output directory in HDFS for your SMOOTHED chinese NB model
!hdfs dfs -rm -r {HDFS_DIR}/nbModelSmooth-output

Deleted /user/root/HW2/nbModelSmooth-output


In [64]:
# part e - write your hadoop streaming job
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer_smooth.py \
  -mapper train_mapper.py \
  -reducer train_reducer_smooth.py \
  -input {HDFS_DIR}/chineseTrain.txt \
  -output {HDFS_DIR}/nbModelSmooth-output \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob6746295891128481139.jar tmpDir=null
19/09/16 18:27:43 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/16 18:27:43 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/16 18:27:44 INFO mapred.FileInputFormat: Total input paths to process : 1
19/09/16 18:27:44 INFO mapreduce.JobSubmitter: number of splits:2
19/09/16 18:27:44 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1567779136510_0048
19/09/16 18:27:44 INFO impl.YarnClientImpl: Submitted application application_1567779136510_0048
19/09/16 18:27:44 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1567779136510_0048/
19/09/16 18:27:44 INFO mapreduce.Job: Running job: job_1567779136510_0048
19/09/16 18:27:53 INFO mapreduce.Job: Job job_1567779136510_0048 running in uber mode : false
19/09/16 18:27:53 INFO mapreduce.Job:  map 0% reduce 0%
19/09

In [65]:
# part e - extract your results (i.e. model) to a local file
!hdfs dfs -cat {HDFS_DIR}/nbModelSmooth-output/part-000* > NaiveBayes/SmoothNBModel.txt
!cat NaiveBayes/SmoothNBModel.txt | column -t

beijing      0.0,1.0,0.1111111111111111,0.14285714285714285
chinese      1.0,5.0,0.2222222222222222,0.42857142857142855
japan        1.0,0.0,0.2222222222222222,0.07142857142857142
macao        0.0,1.0,0.1111111111111111,0.14285714285714285
shanghai     0.0,1.0,0.1111111111111111,0.14285714285714285
tokyo        1.0,0.0,0.2222222222222222,0.07142857142857142
ClassPriors  1.0,3.0,0.25,0.75


# Question 9: Enron Ham/Spam NB Classifier & Results.

Fantastic work. We're finally ready to perform Spam Classification on the Enron Corpus. In this question you'll run the analysis you've developed, report its performance, and draw some conclusions.

### Q9 Tasks:
* __a) train/test split:__ Run the provided code to split our Enron file into a training set and testing set then load them into HDFS. 

[`NOTE:` _Make sure you re calculate the vocab size for just the training set!_]

* __b) train 2 models:__ Write Hadoop Streaming jobs to train MNB Models on the training set with and without smoothing. Save your models to local files at __`NaiveBayes/Unsmoothed/NBmodel.txt`__ and __`NaiveBayes/Smoothed/NBmodel.txt`__. [`NOTE:` _This naming is important because we wrote our classification task so that it expects a file of that name... if this inelegance frustrates you there is an alternative that would involve a few adjustments to your code [read more about it here](http://www.tnoda.com/blog/2013-11-23)._] Finally run the checks that we provide to confirm that your results are correct.


* __c) code:__ Recall that we designed our classification job with just a mapper. An efficient way to report the performance of our models would be to simply add a reducer phase to this job and compute precision and recall right there. Complete the code in __`NaiveBayes/evaluation_reducer.py`__ and then write Hadoop jobs to evaluate your two models on the test set. Report their performance side by side. [`NOTE:` if you need a refresher on precision, recall and F1-score [Wikipedia](https://en.wikipedia.org/wiki/F1_score) is a good resource.]


* __d) short response:__ Compare the performance of your two models. What do you notice about the unsmoothed model's predictions? Can you guess why this is happening? Which evaluation measure do you think is most relevant in our use case? [`NOTE:` _Feel free to answer using your common sense but if you want more information on evaluating the classification task checkout_ [this blogpost](https://tryolabs.com/blog/2013/03/25/why-accuracy-alone-bad-measure-classification-tasks-and-what-we-can-do-about-it/
) or [this paper](http://www.flinders.edu.au/science_engineering/fms/School-CSEM/publications/tech_reps-research_artfcts/TRRA_2007.pdf
)]


* __e) code + short response:__ Let's look at the top ten words with the highest conditional probability in `Spam` and in `Ham`. We'll do this by writing a Hadoop job that sorts the model file (`NaiveBayes/Smoothed/NBmodel.py`). Normally we'd have to run two jobs -- one that sorts on $P(word|ham)$ and another that sorts on $P(word|spam)$. However if we slighly modify the data format in the model file then we can get the top words in each class with just one job. We've written a mapper that will do just this for you. Read through __`NaiveBayes/model_sort_mapper.py`__ and then briefly explain how this mapper will allow us to partition and sort our model file. Write a Hadoop job that uses our mapper and `/bin/cat` for a reducer to partition and sort. Print out the top 10 words in each class (where 'top' == highest conditional probability).[`HINT:` _this should remind you a lot of what we did in Question 6._]


* __f) short response:__ What do you notice about the 'top words' we printed in `e`? How would increasing the smoothing parameter 'k' affect the probabilities for the top words that you identified for 'e'. How would they affect the probabilities of words that occur much more in one class than another? In summary, how does the smoothing parameter 'k' affect the bias and the variance of our model. [`NOTE:` _you do not need to code anything for this task, but if you are struggling with it you could try changing 'k' and see what happens to the test set. We don't recommend doing this exploration with the Enron data because it will be harder to see the impact with such a big vocabulary_]

### Q9 Student Answers:
> __d)__ A lot of the unsmoothed results end up with `-inf` probability for both Spam and Ham classes, which makes it hard to evaluate exactly which class to be predict. I think it is caused by words in test data that are unseen in train data. Without smoothing, the conditional probabilities would be zero, which leads to `-inf` for the log probabilities in our prediction results. In our case, I think the counts of TP, TN, FP and FN are most relevant, because from these counts we can see that the unsmoothed model seems to always predict 'negative' (TN and FN numbers are high). Recall that in our `classify_mapper.py`, we set the prediction rule as `if logpHam >= logpSpam: pred_class = 0; else: pred_class = 1`, since we have a lot of `-inf` in our log probabilities in our test cases based off unsmoothed model, most documents would be end up as negative in our predictions.  

> __e)__ Please note that the __NaiveBayes/model_sort_mapper.py__ has been modified to fix bugs in the results. This mapper adds two new fields to our model file: the first is the predicted class (`maxClass`), and the second one is the higher conditional probability (`maxClassP`). This mapper partitions on the predicted class such that 'ham' records go to one reducer and 'spam' records go to the other one. Then the hadoop job uses the second field `maxClassP` to sort the records within each reducer.   

> __f)__ I notice that the 'top words' printed in __e__ for both classes are some common words from which we cannot really tell the whether it is a spam or not. Increasing the smoothing parameter k will generally cause the conditional probabilities in each class to be closer to each other. Therefore, it will have a larger effect on words that occcured much more in one class than the other. In other words increasing k reduces the variance of our model and thus make our model more generalizable. However, it could increase the bias of our model because as k increases our predictions are more likely to reflect the Class Priors.

__Test/Train split__

In [66]:
# part a - test/train split (RUN THIS CELL AS IS)
!head -n 80 data/enronemail_1h.txt > data/enron_train.txt
!tail -n 20 data/enronemail_1h.txt > data/enron_test.txt
!hdfs dfs -copyFromLocal data/enron_train.txt {HDFS_DIR}
!hdfs dfs -copyFromLocal data/enron_test.txt {HDFS_DIR}

copyFromLocal: `/user/root/HW2/enron_train.txt': File exists
copyFromLocal: `/user/root/HW2/enron_test.txt': File exists


__Training__ (Enron MNB Model _without smoothing_ )

In [67]:
# part b -  Unsmoothed model (FILL IN THE MISSING CODE BELOW)

# clear the output directory
!hdfs dfs -rm -r {HDFS_DIR}/enron-model

# hadoop command
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer.py \
  -mapper train_mapper.py \
  -reducer train_reducer.py \
  -input {HDFS_DIR}/enron_train.txt \
  -output {HDFS_DIR}/enron-model \
  -cmdenv PATH={PATH}

# save the model locally
!mkdir NaiveBayes/Unsmoothed
!hdfs dfs -cat {HDFS_DIR}/enron-model/part-000* > NaiveBayes/Unsmoothed/NBmodel.txt

Deleted /user/root/HW2/enron-model
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob7868480982661916812.jar tmpDir=null
19/09/16 18:44:39 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/16 18:44:40 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/16 18:44:41 INFO mapred.FileInputFormat: Total input paths to process : 1
19/09/16 18:44:41 INFO mapreduce.JobSubmitter: number of splits:2
19/09/16 18:44:41 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1567779136510_0049
19/09/16 18:44:41 INFO impl.YarnClientImpl: Submitted application application_1567779136510_0049
19/09/16 18:44:41 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1567779136510_0049/
19/09/16 18:44:41 INFO mapreduce.Job: Running job: job_1567779136510_0049
19/09/16 18:44:50 INFO mapreduce.Job: Job job_1567779136510_0049 running in uber mode : false
19/09/16 18:44:50 INFO map

In [68]:
# part b - check your UNSMOOTHED model results (RUN THIS CELL AS IS)
!grep assistance NaiveBayes/Unsmoothed/NBmodel.txt
# EXPECTED OUTPUT: assistance	2,4,0.000172547666293,0.000296823983378

assistance	2.0,4.0,0.0001725476662928134,0.00029682398337785694


In [69]:
# part b - check your UNSMOOTHED model results (RUN THIS CELL AS IS)
!grep money NaiveBayes/Unsmoothed/NBmodel.txt
# EXPECTED OUTPUT: money	1,22,8.62738331464e-05,0.00163253190858

money	1.0,22.0,8.62738331464067e-05,0.001632531908578213


__Training__ (Enron MNB Model _with Laplace +1 smoothing_ )

In [72]:
# part b -  Smoothed model (FILL IN THE MISSING CODE BELOW)

# clear the output directory
!hdfs dfs -rm -r {HDFS_DIR}/smooth-model

# hadoop command
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer_smooth.py \
  -mapper train_mapper.py \
  -reducer train_reducer_smooth.py \
  -input {HDFS_DIR}/enron_train.txt \
  -output {HDFS_DIR}/smooth-model \
  -cmdenv PATH={PATH}

# save the model locally
!mkdir NaiveBayes/Smoothed
!hdfs dfs -cat {HDFS_DIR}/smooth-model/part-000* > NaiveBayes/Smoothed/NBmodel.txt

Deleted /user/root/HW2/smooth-model
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob8244645361734375559.jar tmpDir=null
19/09/16 18:50:35 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/16 18:50:35 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/16 18:50:36 INFO mapred.FileInputFormat: Total input paths to process : 1
19/09/16 18:50:37 INFO mapreduce.JobSubmitter: number of splits:2
19/09/16 18:50:37 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1567779136510_0051
19/09/16 18:50:37 INFO impl.YarnClientImpl: Submitted application application_1567779136510_0051
19/09/16 18:50:37 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1567779136510_0051/
19/09/16 18:50:37 INFO mapreduce.Job: Running job: job_1567779136510_0051
19/09/16 18:50:45 INFO mapreduce.Job: Job job_1567779136510_0051 running in uber mode : false
19/09/16 18:50:45 INFO ma

In [73]:
# part b - check your SMOOTHED model results (RUN THIS CELL AS IS)
!grep assistance NaiveBayes/Smoothed/NBmodel.txt
# EXPECTED OUTPUT: assistance	2,4,0.000185804533631,0.000277300205202

assistance	2.0,4.0,0.0001858045336306206,0.00027730020520215184


In [74]:
# part b - check your SMOOTHED model results (RUN THIS CELL AS IS)
!grep money NaiveBayes/Smoothed/NBmodel.txt
# EXPECTED OUTPUT: money	1,22,0.000123869689087,0.00127558094393

money	1.0,22.0,0.0001238696890870804,0.0012755809439298986


__Evaluation__

In [75]:
# part c - write your code in NaiveBayes/evaluation_reducer.py then RUN THIS
!chmod a+x NaiveBayes/evaluation_reducer.py

In [76]:
# part c - unit test your evaluation job on the chinese model (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py 
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py | NaiveBayes/evaluation_reducer.py

d5	1	-8.90668134500626	-8.10769031284611	1
d6	1	-5.780743515794329	-4.179502370564408	1
d7	0	-6.591673732011658	-7.511706880737812	0
d8	0	-4.394449154674438	-5.565796731681498	0
d5	1	-8.90668134500626	-8.10769031284611	 True
d6	1	-5.780743515794329	-4.179502370564408	 True
d7	0	-6.591673732011658	-7.511706880737812	 True
d8	0	-4.394449154674438	-5.565796731681498	 True
Total # Documents: 	4.0
True Positives: 	2.0
True Negatives: 	2.0
False Positives: 	0.0
False Negatives: 	0.0
Accuracy: 	1.0
Precision: 	1.0
Recall: 	1.0
F-Score: 	1.0


In [77]:
# part c - Evaluate the UNSMOOTHED Model Here (FILL IN THE MISSING CODE)

# clear output directory
!hdfs dfs -rm -r {HDFS_DIR}/unsmooth-results

# hadoop job
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/classify_mapper.py,NaiveBayes/evaluation_reducer.py,NaiveBayes/Unsmoothed/NBmodel.txt \
  -mapper classify_mapper.py \
  -reducer evaluation_reducer.py \
  -input {HDFS_DIR}/enron_test.txt \
  -output {HDFS_DIR}/unsmooth-results \
  -cmdenv PATH={PATH}

# retrieve results locally
!hdfs dfs -cat {HDFS_DIR}/unsmooth-results/part-000* > NaiveBayes/Unsmoothed/results.txt


Deleted /user/root/HW2/unsmooth-results
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob7657827792592327386.jar tmpDir=null
19/09/16 18:53:22 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/16 18:53:22 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/16 18:53:24 INFO mapred.FileInputFormat: Total input paths to process : 1
19/09/16 18:53:24 INFO mapreduce.JobSubmitter: number of splits:2
19/09/16 18:53:24 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1567779136510_0052
19/09/16 18:53:24 INFO impl.YarnClientImpl: Submitted application application_1567779136510_0052
19/09/16 18:53:24 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1567779136510_0052/
19/09/16 18:53:24 INFO mapreduce.Job: Running job: job_1567779136510_0052
19/09/16 18:53:33 INFO mapreduce.Job: Job job_1567779136510_0052 running in uber mode : false
19/09/16 18:53:33 INF

In [78]:
# part c - Evaluate the SMOOTHED Model Here (FILL IN THE MISSING CODE)

# clear output directory
!hdfs dfs -rm -r {HDFS_DIR}/smooth-results

# hadoop job
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/classify_mapper.py,NaiveBayes/evaluation_reducer.py,NaiveBayes/Smoothed/NBmodel.txt \
  -mapper classify_mapper.py \
  -reducer evaluation_reducer.py \
  -input {HDFS_DIR}/enron_test.txt \
  -output {HDFS_DIR}/smooth-results \
  -cmdenv PATH={PATH}

# retrieve results locally
!hdfs dfs -cat {HDFS_DIR}/smooth-results/part-000* > NaiveBayes/Smoothed/results.txt

Deleted /user/root/HW2/smooth-results
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob3233415577370136875.jar tmpDir=null
19/09/16 18:54:15 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/16 18:54:16 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/16 18:54:17 INFO mapred.FileInputFormat: Total input paths to process : 1
19/09/16 18:54:17 INFO mapreduce.JobSubmitter: number of splits:2
19/09/16 18:54:17 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1567779136510_0053
19/09/16 18:54:17 INFO impl.YarnClientImpl: Submitted application application_1567779136510_0053
19/09/16 18:54:17 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1567779136510_0053/
19/09/16 18:54:17 INFO mapreduce.Job: Running job: job_1567779136510_0053
19/09/16 18:54:25 INFO mapreduce.Job: Job job_1567779136510_0053 running in uber mode : false
19/09/16 18:54:25 INFO 

In [79]:
# part c - display results 
# NOTE: feel free to modify the tail commands to match the format of your results file
print('=========== UNSMOOTHED MODEL ============')
!tail -n 9 NaiveBayes/Unsmoothed/results.txt
print('=========== SMOOTHED MODEL ============')
!tail -n 9 NaiveBayes/Smoothed/results.txt

=========== UNSMOOTHED MODEL ============
Total # Documents: 	20.0
True Positives: 	1.0
True Negatives: 	9.0
False Positives: 	0.0
False Negatives: 	10.0
Accuracy: 	0.5
Precision: 	1.0
Recall: 	0.09090909090909091
F-Score: 	0.16666666666666669
=========== SMOOTHED MODEL ============
Total # Documents: 	20.0
True Positives: 	11.0
True Negatives: 	6.0
False Positives: 	3.0
False Negatives: 	0.0
Accuracy: 	0.85
Precision: 	0.7857142857142857
Recall: 	1.0
F-Score: 	0.88


__`EXPECTED RESULTS:`__ 
<table>
<th>Unsmoothed Model</th>
<th>Smoothed Model</th>
<tr>
<td><pre>
# Documents:	20
True Positives:	1
True Negatives:	9
False Positives:	0
False Negatives:	10
Accuracy	0.5
Precision	1.0
Recall	0.0909
F-Score	0.1666
</pre></td>
<td><pre>
# Documents:	20
True Positives:	11
True Negatives:	6
False Positives:	3
False Negatives:	0
Accuracy	0.85
Precision	0.7857
Recall	1.0
F-Score	0.88
</pre></td>
</tr>
</table>

__`NOTE:`__ _Don't be too disappointed if these seem low to you. We've trained and tested on a very very small corpus... bigger datasets coming soon!_

__`part e starts here:`__

In [80]:
# part e - write your Hadoop job here (sort smoothed model on P(word|class))

# clear output directory
!hdfs dfs -rm -r {HDFS_DIR}/smooth-sort-output

# hadoop job
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=4 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keypartitioner.options="-k2,2" \
  -D mapreduce.partition.keycomparator.options="-k3,3nr" \
  -files NaiveBayes/model_sort_mapper.py \
  -mapper model_sort_mapper.py \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -input {HDFS_DIR}/smooth-model \
  -output {HDFS_DIR}/smooth-sort-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}


Deleted /user/root/HW2/smooth-sort-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob4519484269899639455.jar tmpDir=null
19/09/16 18:55:33 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/16 18:55:34 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/16 18:55:35 INFO mapred.FileInputFormat: Total input paths to process : 1
19/09/16 18:55:35 INFO mapreduce.JobSubmitter: number of splits:2
19/09/16 18:55:35 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1567779136510_0054
19/09/16 18:55:35 INFO impl.YarnClientImpl: Submitted application application_1567779136510_0054
19/09/16 18:55:35 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1567779136510_0054/
19/09/16 18:55:35 INFO mapreduce.Job: Running job: job_1567779136510_0054
19/09/16 18:55:43 INFO mapreduce.Job: Job job_1567779136510_0054 running in uber mode : false
19/09/16 18:55:43 I

In [81]:
# part e - print top words in each class
for idx in range(2):
    print(f"\n===== part-0000{idx}=====\n")
    !hdfs dfs -cat {HDFS_DIR}/smooth-sort-output/part-0000{idx} | head


===== part-00000=====

ClassPriors	ham	0.5875	47.0,33.0,0.5875,0.4125	
ect	ham	0.023473306082001735	378.0,0.0,0.023473306082001735,5.546004104043037e-05	
and	ham	0.01604112473677691	258.0,277.0,0.01604112473677691,0.015417891409239643	
hou	ham	0.0126347082868822	203.0,0.0,0.0126347082868822,5.546004104043037e-05	
in	ham	0.009971509971509971	160.0,157.0,0.009971509971509971,0.008762686484387999	
for	ham	0.00922829183698749	148.0,153.0,0.00922829183698749,0.008540846320226277	
on	ham	0.007617985878855444	122.0,95.0,0.007617985878855444,0.005324163939881316	
enron	ham	0.007246376811594203	116.0,0.0,0.007246376811594203,5.546004104043037e-05	
i	ham	0.007060572277963582	113.0,106.0,0.007060572277963582,0.00593422439132605	
will	ham	0.007060572277963582	113.0,69.0,0.007060572277963582,0.003882202872830126	
cat: Unable to write to output stream.

===== part-00001=====

the	spam	0.029726581997670677	453.0,535.0,0.02811841942276725,0.029726581997670677	
to	spam	0.023348677278021184	350.0,420.0

### Congratulations, you have completed HW2! Please refer to the readme for submission instructions.

If you would like to provide feedback regarding this homework, please use the survey at: https://docs.google.com/forms/d/1S3P9be24hLspAAPuXINUgMae1v-pUZrmDRmTQ2cwPo0/edit